Aluno: Arlon da Silva Moreira

# NLTK

In [3]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

dataset = pd.read_csv(r'Airline-Sentiment-2-w-AA.csv')
 
conteudoTextual = np.array(dataset['text']) #Selecionado os textos em um array
sentimentos = np.array(dataset['airline_sentiment']) #Sentimento sobre as linhas aéreas

sample_docs = [100, 5817, 7626, 7356, 1008, 7155, 3533, 13010]
sample_data = [(conteudoTextual[index],
                sentimentos[index])
                for index in sample_docs]

lemmatizer = WordNetLemmatizer()
 
def penn_to_wn(tag):

    #Convert between the PennTreebank tags to simple Wordnet tags
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 
def clean_text(text):
    text = text.replace("<br />", " ")
    #text = text.decode("utf-8")
 
    return text
 
def swn_polarity(text):
    
    #Return a sentiment polarity: 0 = negative, 1 = positive  
 
    sentiment = 0.0
    tokens_count = 0
 
    text = clean_text(text)
    
    raw_sentences = sent_tokenize(text)
    
    for raw_sentence in raw_sentences:
     
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
     
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
 
    # judgment call ? Default to positive or negative

    if not tokens_count:
        return 0
 
    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 'positive'
 
    # negative sentiment
    return 'negative'

pred_y = [swn_polarity(text) for text in conteudoTextual]
print(accuracy_score(sentimentos, pred_y))

0.4146857923497268


In [18]:
print(classification_report(sentimentos, pred_y, labels=["positive", "negative"]))

             precision    recall  f1-score   support

   positive       0.21      0.80      0.33      2363
   negative       0.79      0.45      0.58      9178

avg / total       0.67      0.53      0.53     11541



In [6]:
from sklearn.metrics import confusion_matrix
confusion_matrix(sentimentos, pred_y, labels=["positive", "negative"])

array([[1900,  355],
       [4956, 4171]], dtype=int64)

# LIWCSentiment

In [26]:
from nltk import sent_tokenize, word_tokenize
from sklearn.metrics import accuracy_score

dataset = pd.read_csv(r'Airline-Sentiment-2-w-AA.csv')
 
conteudoTextual = np.array(dataset['text'])
sentimentos = np.array(dataset['airline_sentiment'])

positivos = []
negativos = []

for x in range(0, sentimentos.shape[0]):
    if sentimentos[x] == 'positive':
        positivos.append(conteudoTextual[x])
    elif sentimentos[x] == 'negative':
        negativos.append(conteudoTextual[x])
        
db_pos = np.array(positivos)
db_neg = np.array(negativos)

liwc = pd.read_csv(r'LIWC-original.txt',header=None)

palavras = np.array(liwc[0])
sentimentos = np.array(liwc[1])

pos = set()
neg = set()

for i in range(len(sentimentos)):
    if(sentimentos[i]==1):
        pos.add(palavras[i])
    else:
        neg.add(palavras[i])

lemmatizer = WordNetLemmatizer()
 
def check_pol(text):
    
    pol = 0.0

    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        words = word_tokenize(raw_sentence)
        
        for word in words:
            negation = False
            
            if u'não' in text:
                negation = True
                
            if(word in pos):
                pol += 1
 
            if(word in neg):
                pol += -1
            
            if negation:
                pol = -1 * pol
   
    if pol >= 0:
        return 'positive'
 
    return 'negative'

pred_y = [check_pol(text) for text in db_pos]

test_sentiments = np.full(len(pred_y), 'positive')
print('positive acc:', accuracy_score(test_sentiments, pred_y))
print(classification_report(test_sentiments, pred_y))

pred_y = [check_pol(text) for text in db_neg]
test_sentiments = np.full(len(pred_y), 'negative')
print('negative acc:', accuracy_score(test_sentiments, pred_y))
print(classification_report(test_sentiments, pred_y))

positive acc: 0.9991536182818451
             precision    recall  f1-score   support

   negative       0.00      0.00      0.00         0
   positive       1.00      1.00      1.00      2363

avg / total       1.00      1.00      1.00      2363



C:\Users\Arlon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


negative acc: 0.01503595554587056
             precision    recall  f1-score   support

   negative       1.00      0.02      0.03      9178
   positive       0.00      0.00      0.00         0

avg / total       1.00      0.02      0.03      9178

